## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-28-20-50-34 +0000,nypost,Sam’s Club shopper sparks uproar buying all th...,https://nypost.com/2025/11/28/business/sams-cl...
1,2025-11-28-20-47-32 +0000,nyt,Officials Had Been Warned for Over a Year Befo...,https://www.nytimes.com/2025/11/28/world/asia/...
2,2025-11-28-20-44-05 +0000,nypost,Brother of alleged DC gunman Rahmanullah Lakan...,https://nypost.com/2025/11/28/us-news/brother-...
3,2025-11-28-20-42-06 +0000,bbc,Afghans in US issue plea to Trump after Washin...,https://www.bbc.com/news/articles/c0q5w8l88dvo...
4,2025-11-28-20-40-52 +0000,nypost,Wild bodycam footage shows Portland cops hit a...,https://nypost.com/2025/11/28/us-news/bodycam-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2350/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
43,trump,30
45,shooting,23
92,guard,18
135,national,17
20,hong,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
239,2025-11-28-00-57-00 +0000,nypost,Trump claims National Guard shooting suspect ‘...,https://nypost.com/2025/11/27/us-news/trump-cl...,110
49,2025-11-28-18-27-18 +0000,startribune,"In social media posts, Trump targets Somali co...",https://www.startribune.com/in-social-media-po...,107
46,2025-11-28-18-36-00 +0000,wsj,"After D.C. National Guard Shooting, Trump Step...",https://www.wsj.com/politics/policy/trump-push...,103
235,2025-11-28-01-08-59 +0000,wapo,"National Guard member dies after shooting, Tru...",https://www.washingtonpost.com/dc-md-va/2025/1...,102
108,2025-11-28-15-48-03 +0000,nyt,Trump Uses National Guard Shooting to Cast Sus...,https://www.nytimes.com/2025/11/27/us/politics...,93


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
239,110,2025-11-28-00-57-00 +0000,nypost,Trump claims National Guard shooting suspect ‘...,https://nypost.com/2025/11/27/us-news/trump-cl...
39,75,2025-11-28-18-45-45 +0000,nypost,Dog saves retiree from narrowly being killed i...,https://nypost.com/2025/11/28/world-news/elder...
2,55,2025-11-28-20-44-05 +0000,nypost,Brother of alleged DC gunman Rahmanullah Lakan...,https://nypost.com/2025/11/28/us-news/brother-...
100,54,2025-11-28-16-08-10 +0000,nypost,Zelensky’s top aide Andriy Yermak resigns afte...,https://nypost.com/2025/11/28/world-news/zelen...
13,44,2025-11-28-20-24-00 +0000,wsj,President Trump said he was revoking all execu...,https://www.wsj.com/politics/elections/trump-s...
53,44,2025-11-28-18-15-30 +0000,nyt,Israeli Forces Kill Two Palestinians in West B...,https://www.nytimes.com/2025/11/28/world/middl...
37,39,2025-11-28-18-50-28 +0000,latimes,Black Friday shoppers spend more time looking ...,https://www.latimes.com/business/story/2025-11...
31,35,2025-11-28-19-36-14 +0000,nypost,Anna Kepner who died on Carnival cruise ‘fough...,https://nypost.com/2025/11/28/us-news/anna-kep...
101,32,2025-11-28-16-02-34 +0000,nypost,Australian high school student dies after repo...,https://nypost.com/2025/11/28/world-news/year-...
234,32,2025-11-28-01-28-25 +0000,bbc,Carney's new Alberta oil pipeline deal already...,https://www.bbc.com/news/articles/cgkexjxmvd7o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
